In [ ]:
import socceraction as sa
import soccerdata as sd
from scraper.whoscored_chromeless import WhoScored
from pathlib import PosixPath

In [ ]:
# Setup a scraper for the 2021/2022 Premier League season
ws = WhoScored(
    leagues=["GER-Bundesliga"], # "GER-Bundesliga2", "ENG-Premier League", "ESP-La Liga", 
    seasons=[18, 19, 20, 21, 22], # 15, 16, 17, 18, 19, 20, 21, 22, 23
    #no_cache=False,
    #no_store=False,
    data_dir=PosixPath("/home/morten/Develop/Open-Data/soccerdata"),
    # path_to_browser="/usr/bin/chromium",
    #headless=True,
)
api = ws.read_events(output_fmt='loader')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from soccerdata import WhoScored
from socceraction import loader, spadl, xthreat


# Step 2: Initialize Loader for SPADL conversion
ldr = loader.Loader(api, provider="whoscored")  # Use the unified Loader
games = ldr.get_games()
actions = ldr.get_actions()

# Optional: filter to the relevant season
# games = games[games.season_id == 2021]
# actions = actions[actions.game_id.isin(games.game_id)]

# Step 3: Train xT model
filtered = actions[actions["type_name"].isin(["pass", "carry", "dribble"])]
xt_model = xthreat.train(filtered)

# Step 4: Apply xT
filtered["start_xT"], filtered["end_xT"], filtered["xT"] = xthreat.predict(filtered, xt_model)

# Step 5: Plot xT matrix
xt_matrix = xt_model["xT"]
fig, ax = plt.subplots(figsize=(12, 8))
im = ax.imshow(xt_matrix, cmap="plasma", origin="lower",
               extent=[0,120,0,80], aspect="auto")
ax.set_title("Expected Threat (xT) Matrix – 2021/22 Premier League", fontsize=16)
ax.set_xlabel("Pitch X"); ax.set_ylabel("Pitch Y")
cbar = plt.colorbar(im, ax=ax); cbar.set_label("xT value")
plt.show()
